In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests
import lxml

In [2]:
regex_bbc = r"(http(s?):\/\/(www)?[A-Za-z0-9\/\.]+)"
regex_cbc = r"(http(s?):\/\/(www)?[A-Za-z0-9-?=\s\/\.]+)"
xpath_bbc = ['//div[@class="story-body__inner"]/p/text()',
            '//ul/li[@class="tags-list__tags"]/a/text()',
            '//span[contains(@class,"byline__name")]']

# Regex "&amp"
regex_decode = r"(\&amp)"

In [3]:
file_bbc = '../datasets/bbchealth.txt'
columns_name = ['tweetid','date','title']

bbc = pd.read_csv(file_bbc, sep='|', header= None)
bbc.columns = columns_name
bbc.head()

,tweetid,date,title
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised http://bbc.in/...
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll http://bbc...
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater' http://bbc...
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising' http://bb...
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,Coalition 'undermined NHS' - doctors http://bb...


In [4]:
# Split Url out of content
def get_url(regex, sample):
    result = re.search(regex, sample)[0]
    
    return result

# Split text out of content
def split_txt_form_url(regex, sample):
    url = get_url(regex, sample)
    result = sample.strip(url)
    
    return result

# Scrapt content base on xpath command.
def scrapt_w_xpath(xpath_cmd):
    result = dict()
    
    # Content
    contents = xpath_selector.xpath(xpath_cmd[0]).extract()
    result.update({'contents': contents})
    
    # Tags
    tags = xpath_selector.xpath(xpath_cmd[1]).extract()
    result.update({'tags': tags})
    
    # Author
    author = xpath_selector.xpath(xpath_cmd[2]).extract()
    result.update({'author': author})
    
    return result

In [5]:
bbc['url'] = bbc['title'].map(lambda row: get_url(regex_bbc, row))
bbc['title'] = bbc['title'].map(lambda row: split_txt_form_url(regex_bbc, row))
bbc.head()

,tweetid,date,title,url
0,585978391360221184,Thu Apr 09 01:31:50 +0000 2015,Breast cancer risk test devised,http://bbc.in/1CimpJF
1,585947808772960257,Wed Apr 08 23:30:18 +0000 2015,GP workload harming care - BMA poll,http://bbc.in/1ChTBRv
2,585947807816650752,Wed Apr 08 23:30:18 +0000 2015,Short people's 'heart risk greater',http://bbc.in/1ChTANp
3,585866060991078401,Wed Apr 08 18:05:28 +0000 2015,New approach against HIV 'promising',http://bbc.in/1E6jAjt
4,585794106170839041,Wed Apr 08 13:19:33 +0000 2015,oalition 'undermined NHS' - doctors,http://bbc.in/1CnLwK7


In [6]:
bbc_urls = bbc['url'].values

In [7]:
is_attempt = True
for key, url in enumerate(bbc_urls):
    
    try:
        # Start the connection
        res = requests.get(url)

        # Get return content as text
        text_html = res.text

        # Check return header
        if res.status_code == 200:
            print("Header:  ", 200)
    #         soup = BeautifulSoup(res.content, 'lxml')
            xpath_selector = Selector(text=text_html)

            # Scrapt content with XPath
            list_scrapt = scrapt_w_xpath(xpath_bbc)

            count_xp_scrapt = len(list_scrapt)


            if count_xp_scrapt:

                # Add new columns
                if is_attempt:
                    df_temp = pd.DataFrame(columns = list(list_scrapt.keys()))
                    bbc = pd.concat([bbc, df_temp])
                    print(bbc.columns)
                    is_attempt = False

                # Add to dataFrame
                for k_, v_ in list_scrapt.items():
                    if v_:
                        bbc[k_].iloc[key] = v_

            else:
                print("Return list is emptied ! ({})".format(url))
        else:
            print("Nothing is archived because of {} return {} Errors".format(url, res.status_code))
    except:
        print(url)
        pass

Header:   200
Index(['author', 'contents', 'date', 'tags', 'title', 'tweetid', 'url'], dtype='object')


C:\Users\Shaynie\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

C:\Users\Shaynie\Anaconda3\lib\site-packages\pandas\core\indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header

Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header

Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header:   200
Header

http://bbc.in/1s6jkdh
http://bbc.in/1xDUXDx
http://bbc.in/1qqYNwR
http://bbc.in/1qqYNgy
http://bbc.in/1qqxcfq
http://bbc.in/1qpHqws
http://bbc.in/1o5Yxou
http://bbc.in/1lYJ19P
http://bbc.in/1o5SRLh
http://bbc.in/1qoyo2N
http://bbc.in/1o3do2T
http://bbc.in/1qkF7uE
http://bbc.in/1xwsHTb
http://bbc.in/1xwj6vJ
http://bbc.in/1lTqmfq
http://bbc.in/1nZjcuq
http://bbc.in/1nYRwFO
http://bbc.in/1qgUJ2i
http://bbc.in/1lRQB5T
http://bbc.in/1qfLvDA
http://bbc.in/1lQHMtk
http://bbc.in/1rWfuDz
http://bbc.in/1nUwmsy
http://bbc.in/1tjgHqc
http://bbc.in/1nRW4ha
http://bbc.in/1nR6kGy
http://bbc.in/1nQ9471
http://bbc.in/1q6V5IN
http://bbc.in/1q6V7Ah
http://bbc.in/X1lb7M
http://bbc.in/1q4WVK6
http://bbc.in/WWHPxS
http://bbc.in/WWHRpu
http://bbc.in/1tNXJWa
http://bbc.in/1nGsUlh
http://bbc.in/1xfonHU
http://bbc.in/1pX4OkL
http://bbc.in/1lFlf2g
http://bbc.in/1rFY25Z
http://bbc.in/1rDwSN9
http://bbc.in/1tK7iFP
http://bbc.in/1tJwYCo
http://bbc.in/1rCRK7d
http://bbc.in/WOY7sJ
http://bbc.in/1xaaUks
http://bbc.in/

http://bbc.in/1jTJFHW
http://bbc.in/1w5kwzh
http://bbc.in/1jKcTJf
http://bbc.in/1jFUVYv
http://bbc.in/Rkib2U
http://bbc.in/1nauds2
http://bbc.in/1kaHTmh
http://bbc.in/1m129EJ
http://bbc.in/RfAjuH
http://bbc.in/1jA7TXx
http://bbc.in/RfAjuy
http://bbc.in/Rfri4J
http://bbc.in/1jzTACy
http://bbc.in/1lXMbv6
http://bbc.in/1vMN3JH
http://bbc.in/1n4uX1W
http://bbc.in/1lVOleL
http://bbc.in/RctZnA
http://bbc.in/RctXMG
http://bbc.in/1n21OEA
http://bbc.in/1ha11Mp
http://bbc.in/1vIrl9V
http://bbc.in/1k57Jbn
http://bbc.in/1h7MEYV
http://bbc.in/1mWIZm6
http://bbc.in/1jpRNzP
http://bbc.in/SgbX59
http://bbc.in/1mT0MdK
http://bbc.in/1mT0LXu
http://bbc.in/1lKJAEy
http://bbc.in/1lJONfP
http://bbc.in/1lJOO3F
http://bbc.in/1lJOMZr
http://bbc.in/1lJcyVp
http://bbc.in/1vvlcOf
http://bbc.in/1h286yn
http://bbc.in/1h285dH
http://bbc.in/1lHk6Ia
http://bbc.in/Saw4Sf
http://bbc.in/1jYUVDl
http://bbc.in/1gZBpll
http://bbc.in/1lDwJUH
http://bbc.in/1vqtuH2
http://bbc.in/1gXSegn
http://bbc.in/1vql6qZ
http://bbc.in/1j8y

http://bbc.in/1jOYZD9
http://bbc.in/1jOZ1Ll
http://bbc.in/1p94Pmj
http://bbc.in/1p94Pmg
http://bbc.in/OWcpDR
http://bbc.in/1rB2vGM
http://bbc.in/1hsmv6o
http://bbc.in/1dNJd8a
http://bbc.in/1dqDBFL
http://bbc.in/1j2l2rF
http://bbc.in/1gp2u4r
http://bbc.in/1hq6vBJ
http://bbc.in/1goqWms
http://bbc.in/1dM3pXO
http://bbc.in/1rxi6Hh
http://bbc.in/1jDZwYs
http://bbc.in/1jD4zZl
http://bbc.in/1jD4CUW
http://bbc.in/1dnr1ak
http://bbc.in/1p0ttFC
http://bbc.in/1dKoH8r
http://bbc.in/1hkUYUr
http://bbc.in/1glBy5u
http://bbc.in/1glBy5q
http://bbc.in/1kZR0Vm
http://bbc.in/1rruZmo
http://bbc.in/1rruYP8
http://bbc.in/OPvkQI
http://bbc.in/1iWkMKO
http://bbc.in/1hfTvhT
http://bbc.in/1hfTv1F
http://bbc.in/1ro0uxy
http://bbc.in/1rn9djF
http://bbc.in/1hfa5i1
http://bbc.in/1rm89wk
http://bbc.in/1he60uu
http://bbc.in/1ghe55v
http://bbc.in/1ggTCO7
http://bbc.in/1oPcV38
http://bbc.in/OJGtm2
http://bbc.in/OJGtm1
http://bbc.in/1jljoiY
http://bbc.in/1dzTdSi
http://bbc.in/1gaoTCi
http://bbc.in/1dyKEad
http://bbc.in/

http://bbc.in/1hhDgTs
http://bbc.in/1mEz7wN
http://bbc.in/1hgnice
http://bbc.in/1mDdKfk
http://bbc.in/1hgnkkk
http://bbc.in/1mDdHAf
http://bbc.in/LBNBiJ
http://bbc.in/1cQVN4O
http://bbc.in/1cQVN4E
http://bbc.in/1helOiA
http://bbc.in/1bPcn9m
http://bbc.in/KymAfP
http://bbc.in/1jtHA3y
http://bbc.in/1mzNvX7
http://bbc.in/1mzNvGL
http://bbc.in/1ij3ubY
http://bbc.in/1ij3ubP
http://bbc.in/Kx7zuC
http://bbc.in/1myVd3M
http://bbc.in/1hcDbAx
http://bbc.in/1mxF9iI
http://bbc.in/Lyrauz
http://bbc.in/1bHhRms
http://bbc.in/Lyrd9T
http://bbc.in/1jmQTCd
http://bbc.in/LuoNZQ
http://bbc.in/1dMCzBM
http://bbc.in/LuoPAV
http://bbc.in/1jm7nuo
http://bbc.in/KoU5kC
http://bbc.in/1cGIuDO
http://bbc.in/1dFKlgC
http://bbc.in/1i7WbDR
http://bbc.in/KchTYV
http://bbc.in/Kc3ADF
http://bbc.in/1gVTIsh
http://bbc.in/LfGZGA
http://bbc.in/LfGYSZ
http://bbc.in/1mfB174
http://bbc.in/1dz0oNg
http://bbc.in/1dxt2hS
http://bbc.in/1dx45TQ
http://bbc.in/1dwH7vQ
http://bbc.in/1mavRJt
http://bbc.in/1gPDxNo
http://bbc.in/1j06bMV


http://bbc.in/1agNQZd
http://bbc.in/1ah0EPm
http://bbc.in/1dFHWAG
http://bbc.in/18CNDvC
http://bbc.in/1dBiFHP
http://bbc.in/1dBiCMq
http://bbc.in/1cxXuFz
http://bbc.in/18BC3AJ
http://bbc.in/1bLI800
http://bbc.in/18BaaZJ
http://bbc.in/18AV46z
http://bbc.in/1eVPymD
http://bbc.in/1cvRvAZ
http://bbc.in/1aainrO
http://bbc.in/1cuatrQ
http://bbc.in/1eSs55B
http://bbc.in/1eRzQsE
http://bbc.in/1bHGEE5
http://bbc.in/1csbJvD
http://bbc.in/1eQ8WRY
http://bbc.in/1dt3FvD
http://bbc.in/1dsT8kf
http://bbc.in/1a740Eu
http://bbc.in/1a7032B
http://bbc.in/1bFdBkH
http://bbc.in/1bF8X67
http://bbc.in/1cWlFSQ
http://bbc.in/1cpH6Xx
http://bbc.in/1dpU494
http://bbc.in/1bDaIRc
http://bbc.in/1eKXze2
http://bbc.in/1bCviB7
http://bbc.in/1cT5hTi
http://bbc.in/1a2DQTf
http://bbc.in/1brWcgT
http://bbc.in/1brWcgQ
http://bbc.in/1bzN4VF
http://bbc.in/1cjUH2N
http://bbc.in/1i5FN51
http://bbc.in/1dlpzkD
http://bbc.in/1dkYOws
http://bbc.in/1dkYMok
http://bbc.in/1dkYMoj
http://bbc.in/1cQpUiY
http://bbc.in/1cQifkE
http://bbc

In [8]:
# test_url = 'http://bbc.in/14GifAt'
# res = requests.get(test_url)
# author = soup.find('span',{'class':'byline__name'}).text
# author

In [9]:
bbc.head()

,author,contents,date,tags,title,tweetid,url
0,"[<span class=""byline__name"">By James Gallagher...",[Scientists have predicted the odds of women d...,Thu Apr 09 01:31:50 +0000 2015,"[Genetics, Cancer, Medical research]",Breast cancer risk test devised,5.859784e+17,http://bbc.in/1CimpJF
1,"[<span class=""byline__name"">By Nick Triggle</s...",[Better funding and more time with patients ar...,Wed Apr 08 23:30:18 +0000 2015,"[Doctors, NHS]",GP workload harming care - BMA poll,5.859478e+17,http://bbc.in/1ChTBRv
2,"[<span class=""byline__name"">By James Gallagher...","[The shorter you are, the greater your risk of...",Wed Apr 08 23:30:18 +0000 2015,"[Genetics, Heart disease, Medical research]",Short people's 'heart risk greater',5.859478e+17,http://bbc.in/1ChTANp
3,"[<span class=""byline__name"">By Smitha Mundasad...",[The first human trial of a new type of HIV th...,Wed Apr 08 18:05:28 +0000 2015,"[HIV & Aids, Medical research]",New approach against HIV 'promising',5.858661e+17,http://bbc.in/1E6jAjt
4,NaN,"[The government has ""undermined and weakened"" ...",Wed Apr 08 13:19:33 +0000 2015,"[NHS, A&E]",oalition 'undermined NHS' - doctors,5.857941e+17,http://bbc.in/1CnLwK7


In [10]:
bbc.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3929 entries, 0 to 3928
Data columns (total 7 columns):
author      610 non-null object
contents    1243 non-null object
date        3929 non-null object
tags        983 non-null object
title       3929 non-null object
tweetid     3929 non-null float64
url         3929 non-null object
dtypes: float64(1), object(6)
memory usage: 245.6+ KB


In [11]:
# cbc content
## //p/text()
## //div[@class='detailSummary']/text() --> Detail Summary

## //h2[@class='deck']/text() --> h2-sub-title
## //div[@class='story']/span --> Content
## //div[@class='source']/p/text() --> Author


## Internal "Comment" link

In [12]:
dictey = dict({'Title':'ewtwewe', 'tweetew':'leoho'})
type(list(dictey.keys()))

list

In [13]:
join_txt(bbc['Contents'][0])

NameError: name 'join_txt' is not defined

In [ ]:
%run -i scripts.py
bbc['Contents'] = bbc['Contents'].map(lambda row: join_txt(row))

In [16]:
bbc.to_csv('./run_data/bbc_scrapt.csv', sep='|', encoding='utf-8')